In [1]:
load.resolver("imagej.public" at "http://maven.imagej.net/content/groups/public")
load.ivy("net.imagej" % "imagej" % "2.0.0-rc-44") // use % since %% adds scala version
load.ivy("de.christophkraemer" % "rhino-script-engine" % "1.1.0")  // since it's not in the classpath with notebooks per default

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT from sonatype-snapshots, using Sun Oct 25 20:00:40 CET 2015

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT from imagej.public, using Sun Oct 25 20:00:40 CET 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT, using Sun Oct 25 20:00:40 CET 2015 and Sun Oct 25 20:00:40 CET 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT, using Sun Oct 25 20:00:40 CET 2015 and Sun Oct 25 20:00:40 CET 2015

	Choosing imagej.public for com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPSHOT from sonatype-snapshots, using Wed Oct 21 11:44:30 CEST 2015

	Unable to reparse com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPS

```java
/*
 * #%L
 * ImgLib2: a general-purpose, multidimensional image processing library.
 * %%
 * Copyright (C) 2009 - 2015 Tobias Pietzsch, Stephan Preibisch, Barry DeZonia,
 * Stephan Saalfeld, Curtis Rueden, Albert Cardona, Christian Dietz, Jean-Yves
 * Tinevez, Johannes Schindelin, Jonathan Hale, Lee Kamentsky, Larry Lindsey, Mark
 * Hiner, Michael Zinsmaier, Martin Horn, Grant Harris, Aivar Grislis, John
 * Bogovic, Steffen Jaensch, Stefan Helfrich, Jan Funke, Nick Perry, Mark Longair,
 * Melissa Linkert and Dimiter Prodanov.
 * %%
 * Redistribution and use in source and binary forms, with or without
 * modification, are permitted provided that the following conditions are met:
 * 
 * 1. Redistributions of source code must retain the above copyright notice,
 *    this list of conditions and the following disclaimer.
 * 2. Redistributions in binary form must reproduce the above copyright notice,
 *    this list of conditions and the following disclaimer in the documentation
 *    and/or other materials provided with the distribution.
 * 
 * THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
 * AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
 * IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
 * ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
 * LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
 * CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
 * SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
 * INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
 * CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
 * ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
 * POSSIBILITY OF SUCH DAMAGE.
 * #L%
 */


/**
 * Use three different interpolators to 10x magnify a small area
 */
public class Example7
{
	public Example7() throws ImgIOException
	{
		// open with ImgOpener using an ArrayImgFactory
		Img< FloatType > img = new ImgOpener().openImg( "DrosophilaWing.tif",
			new FloatType() );

		ImageJFunctions.show( img );

		// create an InterpolatorFactory RealRandomAccessible using nearst neighbor interpolation
		NearestNeighborInterpolatorFactory< FloatType > factory1 =
			new NearestNeighborInterpolatorFactory< FloatType >();

		// create an InterpolatorFactory RealRandomAccessible using linear interpolation
		NLinearInterpolatorFactory< FloatType > factory2 =
			new NLinearInterpolatorFactory< FloatType >();

		// create an InterpolatorFactory RealRandomAccessible using lanczos interpolation
		LanczosInterpolatorFactory< FloatType > factory3 =
			new LanczosInterpolatorFactory< FloatType >();

		// create a RandomAccessible using the factory and views method
		// it is important to extend the image first, the interpolation scheme might
		// grep pixels outside of the boundaries even when locations inside are queried
		// as they integrate pixel information in a local neighborhood - the size of
		// this neighborhood depends on which interpolator is used
		RealRandomAccessible< FloatType > interpolant1 = Views.interpolate(
			Views.extendMirrorSingle( img ), factory1 );
		RealRandomAccessible< FloatType > interpolant2 = Views.interpolate(
			Views.extendMirrorSingle( img ), factory2 );
		RealRandomAccessible< FloatType > interpolant3 = Views.interpolate(
			Views.extendMirrorSingle( img ), factory3 );

		// define the area in the interpolated image
		double[] min = new double[]{ 105.12, 40.43 };
		double[] max = new double[]{ 129.56, 74.933 };

		FinalRealInterval interval = new FinalRealInterval( min, max );

		ImageJFunctions.show( magnify( interpolant1, interval,
			new ArrayImgFactory< FloatType >(), 10 ) ).setTitle( "Nearest Neighbor Interpolation" );
		ImageJFunctions.show( magnify( interpolant2, interval,
			new ArrayImgFactory< FloatType >(), 10 ) ).setTitle( "Linear Interpolation" );
		ImageJFunctions.show( magnify( interpolant3, interval,
			new ArrayImgFactory< FloatType >(), 10 ) ).setTitle( "Lanczos Interpolation" );
	}

	/**
	 * Compute a magnified version of a given real interval
	 *
	 * @param source - the input data
	 * @param interval - the real interval on the source that should be magnified
	 * @param factory - the image factory for the output image
	 * @param magnification - the ratio of magnification
	 * @return - an Img that contains the magnified image content
	 */
	public static < T extends Type< T > > Img< T > magnify( RealRandomAccessible< T > source,
		RealInterval interval, ImgFactory< T > factory, double magnification )
	{
		int numDimensions = interval.numDimensions();

		// compute the number of pixels of the output and the size of the real interval
		long[] pixelSize = new long[ numDimensions ];
		double[] intervalSize = new double[ numDimensions ];

		for ( int d = 0; d < numDimensions; ++d )
		{
			intervalSize[ d ] = interval.realMax( d ) - interval.realMin( d );
			pixelSize[ d ] = Math.round( intervalSize[ d ] * magnification ) + 1;
		}

		// create the output image
		Img< T > output = factory.create( pixelSize, source.realRandomAccess().get() );

		// cursor to iterate over all pixels
		Cursor< T > cursor = output.localizingCursor();

		// create a RealRandomAccess on the source (interpolator)
		RealRandomAccess< T > realRandomAccess = source.realRandomAccess();

		// the temporary array to compute the position
		double[] tmp = new double[ numDimensions ];

		// for all pixels of the output image
		while ( cursor.hasNext() )
		{
			cursor.fwd();

			// compute the appropriate location of the interpolator
			for ( int d = 0; d < numDimensions; ++d )
				tmp[ d ] = cursor.getDoublePosition( d ) / output.realMax( d ) * intervalSize[ d ]
						+ interval.realMin( d );

			// set the position
			realRandomAccess.setPosition( tmp );

			// set the new value
			cursor.get().set( realRandomAccess.get() );
		}

		return output;
	}

}
```



In [2]:
implicit class previewDataset(curSet: net.imagej.Dataset)(implicit ij: net.imagej.ImageJ) {
    /** saves the file to a temporary directory, read it in again as a bytearray and feed it to the jupyter function **/
    def preview() = {
        val tFile = java.io.File.createTempFile("jupy",".jpg")
        ij.scifio().datasetIO().save(curSet,tFile.getPath())
        val imgFIS = new java.io.FileInputStream(tFile.getPath())
        val out = new Array[Byte](imgFIS.getChannel().size().toInt)
        imgFIS.read(out)
        display.jpg(out)
    }
}

defined class previewDataset
import net.imglib2.`type`.numeric.RealType
defined class dsImgPlus

In [3]:
import ij.ImageJ;
import io.scif.img.ImgIOException;
import io.scif.img.ImgOpener;
import net.imglib2.Cursor;
import net.imglib2.FinalRealInterval;
import net.imglib2.RealInterval;
import net.imglib2.RealRandomAccess;
import net.imglib2.RealRandomAccessible;
import net.imglib2.img.Img;
import net.imglib2.img.ImgFactory;
import net.imglib2.img.array.ArrayImgFactory;
import net.imglib2.img.display.imagej.ImageJFunctions;
import net.imglib2.interpolation.randomaccess.LanczosInterpolatorFactory;
import net.imglib2.interpolation.randomaccess.NLinearInterpolatorFactory;
import net.imglib2.interpolation.randomaccess.NearestNeighborInterpolatorFactory;
import net.imglib2.`type`.Type;
import net.imglib2.`type`.numeric.real.FloatType;
import net.imglib2.util.Util;
import net.imglib2.view.Views;

import ij.ImageJ
import io.scif.img.ImgIOException
import io.scif.img.ImgOpener
import net.imglib2.Cursor
import net.imglib2.FinalRealInterval
import net.imglib2.RealInterval
import net.imglib2.RealRandomAccess
import net.imglib2.RealRandomAccessible
import net.imglib2.img.Img
import net.imglib2.img.ImgFactory
import net.imglib2.img.array.ArrayImgFactory
import net.imglib2.img.display.imagej.ImageJFunctions
import net.imglib2.interpolation.randomaccess.LanczosInterpolatorFactory
import net.imglib2.interpolation.randomaccess.NLinearInterpolatorFactory
import net.imglib2.interpolation.randomaccess.NearestNeighborInterpolatorFactory
import net.imglib2.`type`.Type
import net.imglib2.`type`.numeric.real.FloatType
import net.imglib2.util.Util
import net.imglib2.view.Views

In [4]:
/**
 * Compute a magnified version of a given real interval
 *
 * @param source - the input data
 * @param interval - the real interval on the source that should be magnified
 * @param factory - the image factory for the output image
 * @param magnification - the ratio of magnification
 * @return - an Img that contains the magnified image content
 */
def magnify[T <: Type[T]](source: RealRandomAccessible[T],
    interval: RealInterval, factory: ImgFactory[T], magnification: Double ): Img[T] = 
{
    val numDimensions = interval.numDimensions()

    // compute the number of pixels of the output and the size of the real interval
    val pixelSize = new Array[Long](numDimensions)
    val intervalSize = new Array[Double](numDimensions)

    for(d <- 0 until numDimensions) {
        intervalSize(d) = interval.realMax( d ) - interval.realMin( d )
        pixelSize(d) = Math.round( intervalSize(d) * magnification ) + 1
    }

    // create the output image
    val output = factory.create( pixelSize, source.realRandomAccess().get() );

    // cursor to iterate over all pixels
    val cursor = output.localizingCursor();

    // create a RealRandomAccess on the source (interpolator)
    val realRandomAccess = source.realRandomAccess();

    // the temporary array to compute the position
    val tmp = new Array[Double](numDimensions)

    // for all pixels of the output image
    while ( cursor.hasNext() )
    {
        cursor.fwd();

        // compute the appropriate location of the interpolator
        for(d <- 0 until numDimensions) {
            tmp(d) = cursor.getDoublePosition( d ) / output.realMax( d ) * intervalSize( d )
                    + interval.realMin( d );
        }

        // set the position
        realRandomAccess.setPosition( tmp );

        // set the new value
        cursor.get().set( realRandomAccess.get() );
    }

    output
}



defined function magnify

In [5]:
val ij = new net.imagej.ImageJ()

ij: net.imagej.ImageJ = net.imagej.ImageJ@b3a1c0c

In [26]:
// open with ImgOpener using an ArrayImgFactory
import net.imglib2.`type`.numeric.integer.UnsignedByteType
val inputImg = "sample-data/starstack.tif"
val cOpener = ij.io().getOpener(inputImg)
val inImg = cOpener.open(inputImg)
//val img = cOpener.openImg("sample-data/starstack.jpg",new FloatType() )
val typedImage = ij.scifio().datasetIO().open(inputImg).typedImg(new UnsignedByteType())

[INFO] Reading IFDs
[INFO] Populating metadata
[INFO] Checking comment style
[WARNING] Expected positive value for PhysicalSizeX; got 0.0
[WARNING] Expected positive value for PhysicalSizeY; got 0.0
[WARNING] Expected positive value for PhysicalSizeX; got 0.0
[WARNING] Expected positive value for PhysicalSizeY; got 0.0
[INFO] Reading IFDs
[INFO] Populating metadata
[INFO] Checking comment style
[WARNING] Expected positive value for PhysicalSizeX; got 0.0
[WARNING] Expected positive value for PhysicalSizeY; got 0.0
[WARNING] Expected positive value for PhysicalSizeX; got 0.0
[WARNING] Expected positive value for PhysicalSizeY; got 0.0


import net.imglib2.`type`.numeric.integer.UnsignedByteType
inputImg: java.lang.String = "sample-data/starstack.tif"
cOpener: org.scijava.io.IOPlugin[?0] = plugin:class io.scif.io.DatasetIOPlugin
inImg: Any = starstack.tif
typedImage: net.imagej.ImgPlus[net.imglib2.type.numeric.integer.UnsignedByteType] = io.scif.img.SCIFIOImgPlus@46f07697

In [19]:
ij.dataset().create(inputImg).preview()

: 

In [6]:

		ImageJFunctions.show( img );

		// create an InterpolatorFactory RealRandomAccessible using nearst neighbor interpolation
		NearestNeighborInterpolatorFactory< FloatType > factory1 =
			new NearestNeighborInterpolatorFactory< FloatType >();

		// create an InterpolatorFactory RealRandomAccessible using linear interpolation
		NLinearInterpolatorFactory< FloatType > factory2 =
			new NLinearInterpolatorFactory< FloatType >();

		// create an InterpolatorFactory RealRandomAccessible using lanczos interpolation
		LanczosInterpolatorFactory< FloatType > factory3 =
			new LanczosInterpolatorFactory< FloatType >();

		// create a RandomAccessible using the factory and views method
		// it is important to extend the image first, the interpolation scheme might
		// grep pixels outside of the boundaries even when locations inside are queried
		// as they integrate pixel information in a local neighborhood - the size of
		// this neighborhood depends on which interpolator is used
		RealRandomAccessible< FloatType > interpolant1 = Views.interpolate(
			Views.extendMirrorSingle( img ), factory1 );
		RealRandomAccessible< FloatType > interpolant2 = Views.interpolate(
			Views.extendMirrorSingle( img ), factory2 );
		RealRandomAccessible< FloatType > interpolant3 = Views.interpolate(
			Views.extendMirrorSingle( img ), factory3 );

		// define the area in the interpolated image
		double[] min = new double[]{ 105.12, 40.43 };
		double[] max = new double[]{ 129.56, 74.933 };

		FinalRealInterval interval = new FinalRealInterval( min, max );

		ImageJFunctions.show( magnify( interpolant1, interval,
			new ArrayImgFactory< FloatType >(), 10 ) ).setTitle( "Nearest Neighbor Interpolation" );
		ImageJFunctions.show( magnify( interpolant2, interval,
			new ArrayImgFactory< FloatType >(), 10 ) ).setTitle( "Linear Interpolation" );
		ImageJFunctions.show( magnify( interpolant3, interval,
			new ArrayImgFactory< FloatType >(), 10 ) ).setTitle( "Lanczos Interpolation" );
	}

: 